***GENERATED CODE FOR obesityclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'}], 'feature': 'LocationAbbr', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '53'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationAbbr'}, {'feature_label': 'LocationAbbr', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationAbbr')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'}], 'feature': 'LocationDesc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Alabama', 'max': 'Wyoming', 'missing': '0', 'distinct': '53'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LocationDesc'}, {'feature_label': 'LocationDesc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LocationDesc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Datasource', 'transformation_label': 'String Indexer'}], 'feature': 'Datasource', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Behavioral Risk Factor Surveillance System', 'max': 'Behavioral Risk Factor Surveillance System', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Datasource'}, {'feature_label': 'Datasource', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Datasource')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Class', 'transformation_label': 'String Indexer'}], 'feature': 'Class', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Obesity / Weight Status', 'max': 'Obesity / Weight Status', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Class'}, {'feature_label': 'Class', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Class')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Topic', 'transformation_label': 'String Indexer'}], 'feature': 'Topic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Obesity / Weight Status', 'max': 'Obesity / Weight Status', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Topic'}, {'feature_label': 'Topic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Topic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Question', 'transformation_label': 'String Indexer'}], 'feature': 'Question', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Percent of adults aged 18 years and older who have obesity', 'max': 'Percent of adults aged 18 years and older who have obesity', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Question'}, {'feature_label': 'Question', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Question')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Unit', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Unit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Unit'}, {'feature_label': 'Data_Value_Unit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Unit')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Value', 'max': 'Value', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Type'}, {'feature_label': 'Data_Value_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Footnote_Symbol', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Footnote_Symbol', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '46', 'mean': '', 'stddev': '', 'min': '~', 'max': '~', 'missing': '454', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Footnote_Symbo...'}, {'feature_label': 'Data_Value_Footnote_Symbol', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Data_Value_Footnote_Symbol')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Data_Value_Footnote', 'transformation_label': 'String Indexer'}], 'feature': 'Data_Value_Footnote', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '46', 'mean': '', 'stddev': '', 'min': 'Data not available because sample size is insufficient.', 'max': 'Data not available because sample size is insufficient.', 'missing': '454', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Data_Value_Footnote'}, {'feature_label': 'Data_Value_Footnote', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Data_Value_Footnote')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sample_Size', 'transformation_label': 'String Indexer'}], 'feature': 'Sample_Size', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '454', 'mean': '493.48', 'stddev': '282.7', 'min': '1,002', 'max': '994', 'missing': '46', 'distinct': '418'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sample_Size'}, {'feature_label': 'Sample_Size', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sample_Size')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Total', 'transformation_label': 'String Indexer'}], 'feature': 'Total', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '16', 'mean': '', 'stddev': '', 'min': 'Total', 'max': 'Total', 'missing': '484', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Total'}, {'feature_label': 'Total', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Total')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Age(years)', 'transformation_label': 'String Indexer'}], 'feature': 'Age(years)', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '115', 'mean': '', 'stddev': '', 'min': '18 - 24', 'max': '65 or older', 'missing': '385', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Age(years)'}, {'feature_label': 'Age(years)', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Age(years)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Education', 'transformation_label': 'String Indexer'}], 'feature': 'Education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '70', 'mean': '', 'stddev': '', 'min': 'College graduate', 'max': 'Some college or technical school', 'missing': '430', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Education'}, {'feature_label': 'Education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '469', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Income', 'transformation_label': 'String Indexer'}], 'feature': 'Income', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '132', 'mean': '', 'stddev': '', 'min': '$15,000 - $24,999', 'max': 'Less than $15,000', 'missing': '368', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Income'}, {'feature_label': 'Income', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Income')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Race/Ethnicity', 'transformation_label': 'String Indexer'}], 'feature': 'Race/Ethnicity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '136', 'mean': '', 'stddev': '', 'min': '2 or more races', 'max': 'Other', 'missing': '364', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Race/Ethnicity'}, {'feature_label': 'Race/Ethnicity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Race/Ethnicity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GeoLocation', 'transformation_label': 'String Indexer'}], 'feature': 'GeoLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '491', 'mean': '', 'stddev': '', 'min': '(13.444304, 144.793731)', 'max': '(64.84507995700051, -147.72205903599973)', 'missing': '9', 'distinct': '52'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GeoLocation'}, {'feature_label': 'GeoLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GeoLocation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ClassID', 'transformation_label': 'String Indexer'}], 'feature': 'ClassID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'OWS', 'max': 'OWS', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ClassID'}, {'feature_label': 'ClassID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ClassID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopicID', 'transformation_label': 'String Indexer'}], 'feature': 'TopicID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'OWS1', 'max': 'OWS1', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopicID'}, {'feature_label': 'TopicID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopicID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'QuestionID', 'transformation_label': 'String Indexer'}], 'feature': 'QuestionID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Q036', 'max': 'Q036', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'QuestionID'}, {'feature_label': 'QuestionID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('QuestionID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DataValueTypeID', 'transformation_label': 'String Indexer'}], 'feature': 'DataValueTypeID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'VALUE', 'max': 'VALUE', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DataValueTypeID'}, {'feature_label': 'DataValueTypeID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DataValueTypeID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategory1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Age (years)', 'max': 'Total', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategory1'}, {'feature_label': 'StratificationCategory1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationCategory1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'}], 'feature': 'Stratification1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$15,000 - $24,999', 'max': 'Total', 'missing': '0', 'distinct': '28'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Stratification1'}, {'feature_label': 'Stratification1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Stratification1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationCategoryId1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationCategoryId1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AGEYR', 'max': 'RACE', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationCategoryId1'}, {'feature_label': 'StratificationCategoryId1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'StratificationCategoryId1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StratificationID1', 'transformation_label': 'String Indexer'}], 'feature': 'StratificationID1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AGEYR1824', 'max': 'RACEWHT', 'missing': '0', 'distinct': '28'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StratificationID1'}, {'feature_label': 'StratificationID1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StratificationID1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run obesityclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	finalobesityexcelfile = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Final Obesity Excel file .csv', 'filename': 'Final Obesity Excel file .csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(finalobesityexcelfile)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run obesityclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	obesityclassificationautofe = TransformationMain.run(finalobesityexcelfile,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearStart", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2019.0", "stddev": "0.0", "min": "2019", "max": "2019", "missing": "0"}, "updatedLabel": "YearStart"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearEnd", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2019.0", "stddev": "0.0", "min": "2019", "max": "2019", "missing": "0"}, "updatedLabel": "YearEnd"}, {"transformationsData": [{"feature_label": "LocationAbbr", "transformation_label": "String Indexer"}], "feature": "LocationAbbr", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "53"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationAbbr"}, {"transformationsData": [{"feature_label": "LocationDesc", "transformation_label": "String Indexer"}], "feature": "LocationDesc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Alabama", "max": "Wyoming", "missing": "0", "distinct": "53"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LocationDesc"}, {"transformationsData": [{"feature_label": "Datasource", "transformation_label": "String Indexer"}], "feature": "Datasource", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Behavioral Risk Factor Surveillance System", "max": "Behavioral Risk Factor Surveillance System", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Datasource"}, {"transformationsData": [{"feature_label": "Class", "transformation_label": "String Indexer"}], "feature": "Class", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Obesity / Weight Status", "max": "Obesity / Weight Status", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Class"}, {"transformationsData": [{"feature_label": "Topic", "transformation_label": "String Indexer"}], "feature": "Topic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Obesity / Weight Status", "max": "Obesity / Weight Status", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Topic"}, {"transformationsData": [{"feature_label": "Question", "transformation_label": "String Indexer"}], "feature": "Question", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Percent of adults aged 18 years and older who have obesity", "max": "Percent of adults aged 18 years and older who have obesity", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Question"}, {"transformationsData": [{"feature_label": "Data_Value_Unit", "transformation_label": "String Indexer"}], "feature": "Data_Value_Unit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Unit"}, {"transformationsData": [{"feature_label": "Data_Value_Type", "transformation_label": "String Indexer"}], "feature": "Data_Value_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Value", "max": "Value", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Data_Value", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "454", "mean": "32.49", "stddev": "7.13", "min": "4.6", "max": "51.8", "missing": "46"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Data_Value"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Data_Value_Alt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "454", "mean": "32.49", "stddev": "7.13", "min": "4.6", "max": "51.8", "missing": "46"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Alt"}, {"transformationsData": [{"feature_label": "Data_Value_Footnote_Symbol", "transformation_label": "String Indexer"}], "feature": "Data_Value_Footnote_Symbol", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "46", "mean": "", "stddev": "", "min": "~", "max": "~", "missing": "454", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Footnote_Symbo..."}, {"transformationsData": [{"feature_label": "Data_Value_Footnote", "transformation_label": "String Indexer"}], "feature": "Data_Value_Footnote", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "46", "mean": "", "stddev": "", "min": "Data not available because sample size is insufficient.", "max": "Data not available because sample size is insufficient.", "missing": "454", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Data_Value_Footnote"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Low_Confidence_Limit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "454", "mean": "27.93", "stddev": "7.15", "min": "2.0", "max": "47.2", "missing": "46"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Low_Confidence_Limit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "High_Confidence_Limit ", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "454", "mean": "37.64", "stddev": "8.26", "min": "10.3", "max": "67.1", "missing": "46"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "High_Confidence_Limit "}, {"transformationsData": [{"feature_label": "Sample_Size", "transformation_label": "String Indexer"}], "feature": "Sample_Size", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "454", "mean": "493.48", "stddev": "282.7", "min": "1,002", "max": "994", "missing": "46", "distinct": "418"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sample_Size"}, {"transformationsData": [{"feature_label": "Total", "transformation_label": "String Indexer"}], "feature": "Total", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "16", "mean": "", "stddev": "", "min": "Total", "max": "Total", "missing": "484", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Total"}, {"transformationsData": [{"feature_label": "Age(years)", "transformation_label": "String Indexer"}], "feature": "Age(years)", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "115", "mean": "", "stddev": "", "min": "18 - 24", "max": "65 or older", "missing": "385", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age(years)"}, {"transformationsData": [{"feature_label": "Education", "transformation_label": "String Indexer"}], "feature": "Education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "70", "mean": "", "stddev": "", "min": "College graduate", "max": "Some college or technical school", "missing": "430", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Education"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "469", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "Income", "transformation_label": "String Indexer"}], "feature": "Income", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "132", "mean": "", "stddev": "", "min": "$15,000 - $24,999", "max": "Less than $15,000", "missing": "368", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Income"}, {"transformationsData": [{"feature_label": "Race/Ethnicity", "transformation_label": "String Indexer"}], "feature": "Race/Ethnicity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "136", "mean": "", "stddev": "", "min": "2 or more races", "max": "Other", "missing": "364", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Race/Ethnicity"}, {"transformationsData": [{"feature_label": "GeoLocation", "transformation_label": "String Indexer"}], "feature": "GeoLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "491", "mean": "", "stddev": "", "min": "(13.444304, 144.793731)", "max": "(64.84507995700051, -147.72205903599973)", "missing": "9", "distinct": "52"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GeoLocation"}, {"transformationsData": [{"feature_label": "ClassID", "transformation_label": "String Indexer"}], "feature": "ClassID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "OWS", "max": "OWS", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ClassID"}, {"transformationsData": [{"feature_label": "TopicID", "transformation_label": "String Indexer"}], "feature": "TopicID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "OWS1", "max": "OWS1", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopicID"}, {"transformationsData": [{"feature_label": "QuestionID", "transformation_label": "String Indexer"}], "feature": "QuestionID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Q036", "max": "Q036", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "QuestionID"}, {"transformationsData": [{"feature_label": "DataValueTypeID", "transformation_label": "String Indexer"}], "feature": "DataValueTypeID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "VALUE", "max": "VALUE", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DataValueTypeID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "29.97", "stddev": "17.66", "min": "1", "max": "72", "missing": "0"}, "updatedLabel": "LocationID"}, {"transformationsData": [{"feature_label": "StratificationCategory1", "transformation_label": "String Indexer"}], "feature": "StratificationCategory1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Age (years)", "max": "Total", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategory1"}, {"transformationsData": [{"feature_label": "Stratification1", "transformation_label": "String Indexer"}], "feature": "Stratification1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$15,000 - $24,999", "max": "Total", "missing": "0", "distinct": "28"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Stratification1"}, {"transformationsData": [{"feature_label": "StratificationCategoryId1", "transformation_label": "String Indexer"}], "feature": "StratificationCategoryId1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AGEYR", "max": "RACE", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationCategoryId1"}, {"transformationsData": [{"feature_label": "StratificationID1", "transformation_label": "String Indexer"}], "feature": "StratificationID1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AGEYR1824", "max": "RACEWHT", "missing": "0", "distinct": "28"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StratificationID1"}]}))

	#transformationPostExecutionHook(obesityclassificationautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run obesityclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(obesityclassificationautofe, ["YearStart", "YearEnd", "Data_Value", "Data_Value_Alt", "Low_Confidence_Limit", "High_Confidence_Limit ", "LocationID", "LocationAbbr_stringindexer", "LocationDesc_stringindexer", "Datasource_stringindexer", "Class_stringindexer", "Topic_stringindexer", "Question_stringindexer", "Data_Value_Unit_stringindexer", "Data_Value_Type_stringindexer", "Data_Value_Footnote_Symbol_stringindexer", "Data_Value_Footnote_stringindexer", "Sample_Size_stringindexer", "Total_stringindexer", "Education_stringindexer", "Gender_stringindexer", "Income_stringindexer", "Race/Ethnicity_stringindexer", "GeoLocation_stringindexer", "ClassID_stringindexer", "TopicID_stringindexer", "QuestionID_stringindexer", "DataValueTypeID_stringindexer", "StratificationCategory1_stringindexer", "Stratification1_stringindexer", "StratificationCategoryId1_stringindexer", "StratificationID1_stringindexer"], "Age(years)_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

